# 2. Time series analysis

Conduct time-based analysis of user behavior, compare churned versus retained users over time 

(summary)

In [1]:
import pandas as pd
import numpy as np
import pyarrow
import matplotlib.pyplot as plt
import seaborn as sns
import ydata_profiling

In [2]:
data = pd.read_parquet('train.parquet', engine = 'pyarrow')

In [3]:
churn_events = data[data['page'] == 'Cancellation Confirmation'][['userId', 'time']]
churn_events = churn_events.rename(columns={'time': 'churn_time'})

In [4]:
all_users = data['userId'].unique()
churned_users = churn_events['userId'].unique()
non_churn_users = [u for u in all_users if u not in churned_users]

In [5]:
bad_pages = ['Downgrade', 'Thumbs Down', 'Submit Downgrade', 'Roll Advert', 'Error', 'Submit Downgrade']
good_pages = ['Thumbs Up', 'Add to Playlist', 'Add Friend', 'Upgrade', 'Submit Upgrade']
help_pages = ['Help', 'About', 'Settings', 'Save Settings']

In [7]:
def quick_analysis(user_id):
    user_data = data[data['userId'] == user_id].copy()

    #churned
    try:
        churn_time = churn_events.loc[churn_events['userId'] == user_id, 'churn_time'].iloc[0]
        user_data['days_before_churn'] = (churn_time - user_data['time']).dt.days
        user_data['churned'] = 1
        user_data['reference_date'] = churn_time
        
    #not churned
    except (IndexError, KeyError):
        last_dataset_date = data['time'].max()
        user_data['days_before_churn'] = (last_dataset_date - user_data['time']).dt.days
        user_data['churned'] = 0
        user_data['reference_date'] = last_dataset_date
    
    return user_data

In [10]:
sample_users = list(churned_users[:5]) + list(non_churn_users[:5])

for user in sample_users:
    user_data = quick_analysis(user)
    print(f'User {user} had {len(user_data)} "events".')

User 1749042 had 1223 "events".
User 1222580 had 2163 "events".
User 1385500 had 3313 "events".
User 1032628 had 287 "events".
User 1009070 had 414 "events".
User 1563081 had 573 "events".
User 1697168 had 1965 "events".
User 1714398 had 1814 "events".
User 1010522 had 2116 "events".
User 1475659 had 1373 "events".


In [11]:
def analyze_activity_pattern(user_data):
    user_data = user_data.copy()
    
    daily_activity = user_data.groupby('days_before_churn').agg(
    total_events=('page', 'count'),
    songs_played=('page', lambda x: (x == 'NextSong').sum()),
    likes=('page', lambda x: (x == 'Thumbs Up').sum()),
    dislikes=('page', lambda x: (x == 'Thumbs Down').sum()),
    errors=('page', lambda x: (x == 'Error').sum()),
    ads=('page', lambda x: (x == 'Roll Advert').sum()),
    help_about=('page', lambda x: x.isin(help_pages).sum()),
    good_events=('page', lambda x: x.isin(good_pages).sum()),
    bad_events=('page', lambda x: x.isin(bad_pages).sum())
    )
    
    return daily_activity

In [16]:
sample_churner = sample_users[0]
data_churner = quick_analysis(sample_churner)
sample_churner1 = sample_users[1]
data_churner1 = quick_analysis(sample_churner1)
sample_non_churner = sample_users[-1]
data_non_churner = quick_analysis(sample_non_churner)
sample_non_churner1 = sample_users[-2]
data_non_churner1 = quick_analysis(sample_non_churner1)

In [14]:
analyze_activity_pattern(data_churner)

,total_events,songs_played,likes,dislikes,errors,ads,help_about,good_events,bad_events
days_before_churn,,,,,,,,,
0,61,43,3,0,0,0,4,5,2
1,33,27,2,0,0,0,1,2,1
7,230,195,6,0,0,0,3,18,2
8,225,187,12,1,0,0,7,24,4
9,82,68,5,1,0,0,3,9,1
10,50,39,3,0,0,0,2,6,0
12,7,5,1,0,0,0,0,1,0
14,22,17,1,0,0,0,0,2,0
15,209,179,5,4,0,0,2,15,4


In [17]:
analyze_activity_pattern(data_churner1)

,total_events,songs_played,likes,dislikes,errors,ads,help_about,good_events,bad_events
days_before_churn,,,,,,,,,
0,179,143,8,0,1,1,5,16,3
2,140,117,6,0,0,0,1,15,0
3,61,47,3,0,0,3,0,8,3
5,59,50,0,0,0,5,2,0,5
11,5,4,0,0,0,1,0,0,1
12,14,8,0,2,0,0,0,0,2
13,1,0,0,0,0,0,0,0,0
19,203,149,9,3,0,7,2,20,13
20,128,109,3,2,0,0,2,8,2


In [15]:
analyze_activity_pattern(data_non_churner)

,total_events,songs_played,likes,dislikes,errors,ads,help_about,good_events,bad_events
days_before_churn,,,,,,,,,
3,259,218,11,4,0,0,5,19,5
13,272,231,12,3,0,0,6,19,8
17,102,82,6,1,0,1,1,9,4
18,45,37,0,0,0,1,0,6,1
26,5,3,1,0,0,0,0,1,0
28,14,9,2,0,0,0,0,3,0
36,106,83,3,1,0,0,5,10,3
37,103,88,4,1,0,0,1,7,2
39,68,58,1,0,0,0,0,5,0


In [18]:
analyze_activity_pattern(data_non_churner1)

,total_events,songs_played,likes,dislikes,errors,ads,help_about,good_events,bad_events
days_before_churn,,,,,,,,,
2,28,23,0,0,0,4,0,1,4
4,33,22,2,0,0,4,0,4,4
5,77,66,0,0,0,3,3,1,3
6,91,65,5,1,0,2,3,12,3
7,1,1,0,0,0,0,0,0,0
9,40,33,1,1,0,2,0,3,3
11,27,21,0,0,0,2,0,0,2
12,62,50,2,0,0,7,0,4,7
13,54,40,2,1,0,2,2,5,3


In [21]:
def analyze_multiple_users_balanced(n_churners=50, n_non_churners=50):
    all_patterns = []

    churner_ids = churned_users[:n_churners]
    non_churner_ids = non_churn_users[:n_non_churners]
    
    user_ids = list(churner_ids) + list(non_churner_ids)
    
    for user_id in user_ids:
        user_data = quick_analysis(user_id)
        
        if user_data is not None and len(user_data) > 0:
            daily = analyze_activity_pattern(user_data)
            max_days = daily.index.max()
            
            if len(daily) > 0:
                pattern = {
                    'userId': user_id,
                    'churned': 1 if user_data['churned'].iloc[0] else 0,
                    'total_events': len(user_data),
                    'active_days': len(daily),
                    'active_days_ratio': len(daily) / len(daily) / max_days if max_days > 0 else 1,
                    'avg_events_per_day': daily['total_events'].mean(),
                    'song_ratio': daily['songs_played'].sum() / len(user_data),
                    'thumbs_ratio': (daily['likes'].sum() - daily['dislikes'].sum()) / len(user_data),
                    'last_day_activity': daily.loc[0, 'total_events'] if 0 in daily.index else 0,
                    'reference_date': user_data['reference_date'].iloc[0]
                }
                all_patterns.append(pattern)
    
    return pd.DataFrame(all_patterns)

In [22]:
analyze_multiple_users_balanced()

,userId,churned,total_events,active_days,active_days_ratio,avg_events_per_day,song_ratio,thumbs_ratio,last_day_activity,reference_date
0,1749042,1,1223,14,0.050000,87.357143,0.837285,0.034342,61,2018-10-21 01:16:24
1,1222580,1,2163,18,0.034483,120.166667,0.828941,0.031438,179,2018-10-30 23:17:30
2,1385500,1,3313,25,0.021277,132.520000,0.831874,0.032297,188,2018-11-17 04:54:09
3,1032628,1,287,1,1.000000,287.000000,0.864111,0.052265,287,2018-10-01 16:27:17
4,1009070,1,414,3,0.500000,138.000000,0.874396,0.021739,27,2018-10-03 17:21:39
...,...,...,...,...,...,...,...,...,...,...
95,1759311,0,1089,13,0.020408,83.769231,0.851240,0.031221,0,2018-11-20 00:00:00
96,1682050,0,1577,20,0.020408,78.850000,0.812302,0.034876,213,2018-11-20 00:00:00
97,1177341,0,71,2,0.020408,35.500000,0.830986,0.042254,0,2018-11-20 00:00:00
98,1918963,0,2810,22,0.020408,127.727273,0.848754,0.032740,0,2018-11-20 00:00:00


In [23]:
def categorize_gap(gap_days):
    if gap_days <= 0:
        return 'IMMEDIATE'
    elif gap_days <= 1:
        return 'DAY_1'
    elif gap_days <= 3:
        return 'EARLY'  
    elif gap_days <= 7:
        return 'WEEK_1'
    elif gap_days <= 30:
        return 'MONTH_1'
    else:
        return 'LONG_DELAY'  
    
def analyze_registration_gaps_for_users(n_churners=50, n_non_churners=50):
    churner_ids = churned_users[:n_churners]
    non_churner_ids = non_churn_users[:n_non_churners]
    
    user_ids = list(churner_ids) + list(non_churner_ids)

    gaps = {}
    
    for user_id in user_ids:
        user_data = data[data['userId'] == user_id]
        
        if len(user_data) > 0:
            reg_time = user_data['registration'].iloc[0] if 'registration' in user_data.columns else None
            first_activity = user_data['time'].min()
            
            if pd.notna(reg_time) and pd.notna(first_activity):
                gap_days = (first_activity - reg_time).days
                gaps[user_id] = {
                    'reg_time': reg_time,
                    'first_activity': first_activity,
                    'gap_days': gap_days,
                    'gap_category': categorize_gap(gap_days),
                    'churned': 1 if user_id in churn_events['userId'].values else 0
                }
    
    return pd.DataFrame.from_dict(gaps, orient='index')

In [24]:
analyze_registration_gaps_for_users(10, 10)

,reg_time,first_activity,gap_days,gap_category,churned
1749042,2018-08-08 13:22:21,2018-10-01 00:00:01,53,LONG_DELAY,1
1222580,2018-08-16 02:31:00,2018-10-01 00:00:03,45,LONG_DELAY,1
1385500,2018-08-16 04:30:35,2018-10-01 00:00:06,45,LONG_DELAY,1
1032628,2018-09-22 12:15:45,2018-10-01 00:00:06,8,MONTH_1,1
1009070,2018-09-24 11:58:56,2018-10-01 00:00:08,6,WEEK_1,1
1144647,2018-09-08 05:11:14,2018-10-01 00:00:11,22,MONTH_1,1
1834381,2018-09-15 04:00:36,2018-10-01 00:00:12,15,MONTH_1,1
1240184,2018-09-19 18:34:21,2018-10-01 00:00:13,11,MONTH_1,1
1063929,2018-09-21 14:55:56,2018-10-01 00:00:13,9,MONTH_1,1
1559837,2018-09-28 06:39:09,2018-10-01 00:00:14,2,EARLY,1
